In [1]:
#pip install tensorflow==2.2.0

In [2]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from splitters.splitters import SingletaskStratifiedSplitter

from sklearn.ensemble import RandomForestRegressor
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error

import tensorflow as tf
print(tf.version.VERSION)

from parameterOptimization.HyperparameterOpt import HyperparamOpt_Valid, HyperparamOpt_CV


2.2.0


In [3]:
#Load Dataset
dataset = CSVLoader(dataset_path='data/PC-3.csv', 
                    mols_field='smiles', 
                    labels_fields='pIC50')
dataset = dataset.create_dataset()
dataset.get_shape()

Mols_shape:  4294
Features_shape:  X not defined!
Labels_shape:  (4294,)


In [4]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Mols_shape:  4294
Features_shape:  (4294, 1024)
Labels_shape:  (4294,)


In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

train_dataset.get_shape()
valid_dataset.get_shape()
test_dataset.get_shape()

Mols_shape:  2574
Features_shape:  (2574, 1024)
Labels_shape:  (2574,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)


In [6]:
#Scikit-Learn Random Forest
#rf = RandomForestRegressor()
#model = SklearnModel(model=rf)

In [7]:
from models.kerasModels import KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


input_dim = train_dataset.X.shape[1]
print(input_dim)

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=[1024]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

model = KerasModel(build_model, mode='regression', epochs = 5, verbose=1)

1024


In [8]:
#cross validation
model.cross_validate(dataset, Metric(r2_score), folds=3)

Computing K-fold split
Epoch 1/5
287/287 [==============================] - 0s 2ms/step - loss: 1.4657 - mae: 0.8271 - mse: 1.4657
Epoch 2/5
287/287 [==============================] - 0s 2ms/step - loss: 0.4967 - mae: 0.5247 - mse: 0.4967
Epoch 3/5
287/287 [==============================] - 0s 1ms/step - loss: 0.3694 - mae: 0.4568 - mse: 0.3694
Epoch 4/5
287/287 [==============================] - 0s 1ms/step - loss: 0.3006 - mae: 0.4088 - mse: 0.3006
Epoch 5/5
287/287 [==============================] - 0s 2ms/step - loss: 0.2542 - mae: 0.3749 - mse: 0.2542
Train Score: 
287/287 [==============================] - 0s 686us/step
6.55 6.6052065
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
ERROR:  invalid index to scalar variable.


TypeError: object of type 'numpy.float32' has no len()

In [ ]:
# model training
model.fit(train_dataset)

In [ ]:
metrics = [Metric(mean_absolute_error), Metric(mean_squared_error), Metric(median_absolute_error), Metric(r2_score)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)


In [ ]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators=10, max_features='auto', criterion='mse'):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features,
                                     criterion=criterion)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "criterion": ["mse", "mae"]
                  }
    
model = SklearnModel(rf_model_builder, 'regression')

In [ ]:
#Hyperparameter Optimization
#optimizer = HyperparamOpt_Valid(rf_model_builder)

#best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, 
#                                                                     train_dataset, 
#                                                                     valid_dataset, 
#                                                                     Metric(r2_score),
#                                                                     n_iter_search=15)

#print('#################')
#print(best_hyperparams)
#print(best_rf)

In [ ]:
#Evaluate model
#best_rf.evaluate(test_dataset, metrics)

In [ ]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('sklearn',
                                                                     params_dict_rf, 
                                                                     train_dataset,  
                                                                     'r2', 
                                                                     cv=3,
                                                                     n_iter_search=10,
                                                                    n_jobs=8)

print('#################')
print(best_hyperparams)
print(best_rf)

#Evaluate model
best_rf.evaluate(test_dataset, metrics)